# 04. Advanced RAG: Re-ranking & HYDE - 주석 포함 버전

LlamaIndex Reranker 이용

In [ ]:
# ==========================================
# LLMRerank 기반 고급 RAG 시스템 (pandas 오류 해결 버전)
# ==========================================

"""
pandas 순환 임포트 문제 완전 해결 방법:

1. 가상환경 재생성 (권장):
   conda deactivate
   conda create -n rag_env python=3.10
   conda activate rag_env

2. 패키지 순차 설치:
   pip install numpy==1.24.4
   pip install pandas==2.0.3
   pip install llama-index llama-index-llms-openai llama-index-embeddings-openai
   pip install python-dotenv
"""

import os
import sys
import warnings
from pathlib import Path

# 경고 메시지 억제
warnings.filterwarnings('ignore')

# pandas 문제 우회를 위한 환경 설정
os.environ['TOKENIZERS_PARALLELISM'] = 'false'  # 토크나이저 병렬 처리 비활성화

print("🔧 시스템 초기화 중...")

# 환경변수 로드
try:
    from dotenv import load_dotenv
    load_dotenv()
    print("✅ 환경변수 로드 완료")
except ImportError:
    print("⚠️ python-dotenv 미설치. 환경변수를 수동으로 설정하세요.")

# API 키 확인
if not os.getenv("OPENAI_API_KEY"):
    print("❌ OPENAI_API_KEY가 필요합니다.")
    print("💡 .env 파일에 OPENAI_API_KEY=your_key_here 추가")
    sys.exit(1)

# ==========================================
# 안전한 모듈 임포트 (pandas 우회)
# ==========================================

print("📦 필수 모듈 로딩 중...")

try:
    # 기본 LlamaIndex 모듈
    from llama_index.core import Settings, Document
    from llama_index.llms.openai import OpenAI
    from llama_index.embeddings.openai import OpenAIEmbedding
    print("✅ 기본 모듈 로드")
    
    # 벡터 스토어 관련
    from llama_index.core import VectorStoreIndex
    print("✅ 벡터 스토어 모듈 로드")
    
    # 후처리 모듈 (LLMRerank 포함)
    from llama_index.core.postprocessor import LLMRerank
    from llama_index.core.postprocessor import SimilarityPostprocessor
    print("✅ 후처리 모듈 로드")
    
except ImportError as e:
    print(f"❌ 모듈 임포트 실패: {e}")
    print("💡 패키지 재설치가 필요할 수 있습니다.")
    sys.exit(1)

# ==========================================
# 설정 초기화
# ==========================================

print("⚙️ 시스템 설정 중...")

# LLM 설정 (안정적인 모델 사용)
Settings.llm = OpenAI(
    model="gpt-4o-mini",
    temperature=0.1,  # 일관된 응답
    max_tokens=2000,  # 충분한 응답 길이
    timeout=60  # 타임아웃 설정
)

# 임베딩 설정 (가벼운 모델 사용)
Settings.embed_model = OpenAIEmbedding(
    model="text-embedding-3-small",
    dimensions=512,  # 적절한 차원 수
    timeout=60
)

print("✅ 모델 설정 완료")

# ==========================================
# 문서 데이터 생성 (파일 시스템 우회)
# ==========================================

print("📚 문서 데이터 준비 중...")

# 정책 문서 샘플 데이터
policy_documents = [
    {
        "id": "policy_2024_overview",
        "title": "2024년 정책 개요",
        "content": """
        2024년 주요 정책 변화 핵심 사항:

        1. 디지털 전환 정책
        - 공공서비스 디지털화 가속화
        - 민원 처리 온라인 우선 원칙 도입
        - 디지털 접근성 강화 방안 마련

        2. 환경 정책 강화
        - 탄소중립 2050 로드맵 구체화
        - 재생에너지 비중 30% 목표 설정
        - 친환경 교통수단 보급 확대

        3. 사회보장 제도 개편
        - 기본소득 시범사업 확대
        - 노인장기요양보험 적용 범위 확대
        - 청년 주거지원 정책 강화

        4. 교육 정책 혁신
        - AI 기반 맞춤형 교육 도입
        - 평생학습 체계 구축
        - 디지털 역량 교육 의무화
        """
    },
    {
        "id": "policy_2025_changes",
        "title": "2025년 정책 변화",
        "content": """
        2025년 핵심 정책 변화 요약:

        1. AI 거버넌스 체계 구축
        - AI 윤리 가이드라인 법제화
        - AI 영향평가 제도 도입
        - 알고리즘 투명성 의무화

        2. 데이터 경제 활성화
        - 개인정보보호법 개정
        - 데이터 거래소 설립
        - 데이터 주권 강화 정책

        3. 스마트시티 확산
        - 5G 기반 인프라 구축
        - IoT 센서 네트워크 확대
        - 시민참여형 도시계획 도입

        4. 디지털 헬스케어
        - 원격의료 전면 허용
        - 개인건강기록(PHR) 통합 관리
        - AI 기반 진단 지원 시스템 도입

        5. 금융 디지털화
        - 중앙은행 디지털화폐(CBDC) 시범운영
        - 디지털 자산 규제 프레임워크 마련
        - 핀테크 규제 샌드박스 확대
        """
    },
    {
        "id": "implementation_strategy",
        "title": "정책 이행 전략",
        "content": """
        정책 변화 이행 전략 및 핵심 포인트:

        1. 단계적 이행 방안
        - 1단계(2025년 상반기): 법적 기반 마련
        - 2단계(2025년 하반기): 시범사업 실시
        - 3단계(2026년): 전면 시행

        2. 이해관계자 협력 체계
        - 정부-민간 파트너십 강화
        - 시민사회 참여 확대
        - 국제협력 네트워크 구축

        3. 성과 관리 체계
        - KPI 기반 성과 측정
        - 분기별 점검 및 평가
        - 피드백 반영 메커니즘

        4. 위험 관리 방안
        - 정책 실패 리스크 분석
        - 대응 시나리오 사전 준비
        - 모니터링 시스템 구축

        5. 예산 및 재원 조달
        - 정책별 예산 배정 계획
        - 민간투자 유치 방안
        - 효율적 재원 활용 전략
        """
    }
]

# Document 객체 생성 (파일 I/O 없이)
documents = []
for doc_data in policy_documents:
    doc = Document(
        text=doc_data["content"],
        metadata={
            "title": doc_data["title"],
            "doc_id": doc_data["id"]
        }
    )
    documents.append(doc)

print(f"✅ {len(documents)}개 문서 생성 완료")

# ==========================================
# 벡터 인덱스 구축
# ==========================================

print("🔍 벡터 인덱스 생성 중...")

try:
    # 벡터 인덱스 생성
    index = VectorStoreIndex.from_documents(
        documents,
        show_progress=False  # 진행률 표시 비활성화 (안정성)
    )
    print("✅ 벡터 인덱스 생성 완료")
    
except Exception as e:
    print(f"❌ 인덱스 생성 실패: {e}")
    sys.exit(1)

# ==========================================
# 고급 후처리 파이프라인 구성
# ==========================================

print("🚀 고급 후처리 파이프라인 구성 중...")

try:
    # 1. 유사도 기반 필터링
    similarity_filter = SimilarityPostprocessor(
        similarity_cutoff=0.7  # 유사도 임계값
    )
    
    # 2. LLM 기반 리랭킹 (핵심 기능)
    llm_reranker = LLMRerank(
        choice_batch_size=3,  # 배치 크기 (안정성을 위해 작게)
        top_n=3,  # 최종 선택 문서 수
        llm=Settings.llm  # 설정된 LLM 사용
    )
    
    # 후처리 파이프라인 조합
    postprocessors = [similarity_filter, llm_reranker]
    
    print("✅ 후처리 파이프라인 구성 완료")
    
except Exception as e:
    print(f"⚠️ 고급 후처리 설정 실패: {e}")
    print("💡 기본 설정으로 대체합니다.")
    postprocessors = []

# ==========================================
# 쿼리 엔진 생성
# ==========================================

print("⚙️ 쿼리 엔진 생성 중...")

try:
    # 고급 쿼리 엔진 생성
    query_engine = index.as_query_engine(
        similarity_top_k=10,  # 초기 검색 결과 수 (리랭킹 전)
        node_postprocessors=postprocessors,  # 후처리 파이프라인 적용
        response_mode="tree_summarize",  # 응답 생성 방식
        verbose=False  # 상세 로그 비활성화
    )
    print("✅ 쿼리 엔진 생성 완료")
    
except Exception as e:
    print(f"❌ 쿼리 엔진 생성 실패: {e}")
    # 기본 쿼리 엔진으로 폴백
    query_engine = index.as_query_engine(
        similarity_top_k=5,
        response_mode="compact"
    )
    print("✅ 기본 쿼리 엔진으로 대체 완료")

# ==========================================
# 질의응답 함수
# ==========================================

def query_with_details(question: str, show_sources: bool = True):
    """상세한 질의응답 함수"""
    
    print(f"\n🤔 질문: {question}")
    print("=" * 60)
    
    try:
        # 쿼리 실행
        print("🔍 검색 및 분석 중...")
        response = query_engine.query(question)
        
        # 답변 출력
        print("\n💡 답변:")
        print("-" * 40)
        print(str(response))
        
        # 소스 정보 출력 (선택적)
        if show_sources and hasattr(response, 'source_nodes'):
            print(f"\n📚 참조된 문서 ({len(response.source_nodes)}개):")
            print("-" * 40)
            
            for i, node in enumerate(response.source_nodes, 1):
                title = node.metadata.get('title', '제목 없음')
                score = getattr(node, 'score', 'N/A')
                print(f"{i}. {title} (유사도: {score})")
        
        return str(response)
        
    except Exception as e:
        error_msg = f"❌ 쿼리 실행 오류: {e}"
        print(error_msg)
        return error_msg

# ==========================================
# 테스트 실행
# ==========================================

def run_comprehensive_test():
    """종합 테스트 실행"""
    
    test_queries = [
        "정책 변화 핵심만 알려줘",
        "2025년에 새로 도입되는 AI 관련 정책은?",
        "디지털 전환과 관련된 주요 변화점은?",
        "정책 이행을 위한 단계별 계획은?",
        "환경 정책에서 가장 중요한 변화는 무엇인가?"
    ]
    
    print("\n🎯 종합 테스트 시작")
    print("=" * 70)
    
    for i, query in enumerate(test_queries, 1):
        print(f"\n[테스트 {i}/{len(test_queries)}]")
        query_with_details(query, show_sources=True)
        print("\n" + "="*70)
    
    print("✅ 모든 테스트 완료!")

# ==========================================
# 대화형 모드
# ==========================================

def interactive_mode():
    """대화형 질의응답 모드"""
    
    print("\n💬 대화형 모드 (종료: 'quit')")
    print("-" * 40)
    
    while True:
        try:
            question = input("\n질문을 입력하세요: ").strip()
            
            if question.lower() in ['quit', 'exit', '종료']:
                print("👋 대화를 종료합니다.")
                break
            
            if not question:
                continue
            
            query_with_details(question, show_sources=False)
            
        except KeyboardInterrupt:
            print("\n\n👋 대화를 종료합니다.")
            break
        except Exception as e:
            print(f"❌ 오류: {e}")

# ==========================================
# 메인 실행부
# ==========================================

if __name__ == "__main__":
    print("\n🎉 고급 RAG 시스템 준비 완료!")
    print("=" * 50)
    
    # 기본 테스트 실행
    print("\n📋 기본 테스트:")
    query_with_details("정책 변화 핵심만 알려줘")
    
    # 추가 테스트 여부 확인
    print("\n🔄 전체 테스트를 실행하시겠습니까? (y/n): ", end="")
    try:
        if input().strip().lower() in ['y', 'yes', '예']:
            run_comprehensive_test()
    except:
        pass
    
    # 대화형 모드 여부 확인
    print("\n💬 대화형 모드를 시작하시겠습니까? (y/n): ", end="")
    try:
        if input().strip().lower() in ['y', 'yes', '예']:
            interactive_mode()
    except:
        pass
    
    print("\n🎊 프로그램 종료")

# ==========================================
# 추가 유틸리티 함수
# ==========================================

def benchmark_retrieval(query: str, top_k_values: list = [5, 10, 15]):
    """다양한 top_k 값으로 검색 성능 벤치마크"""
    
    print(f"\n🔬 검색 성능 벤치마크: '{query}'")
    print("=" * 60)
    
    for top_k in top_k_values:
        print(f"\n📊 Top-K = {top_k}")
        print("-" * 30)
        
        try:
            # 임시 쿼리 엔진 생성
            temp_qe = index.as_query_engine(
                similarity_top_k=top_k,
                response_mode="compact"
            )
            
            response = temp_qe.query(query)
            print(f"응답 길이: {len(str(response))}자")
            print(f"응답 미리보기: {str(response)[:100]}...")
            
        except Exception as e:
            print(f"❌ 오류: {e}")

# 사용 예시 (필요시 주석 해제):
# benchmark_retrieval("정책 변화 핵심", [3, 5, 10])

🔧 시스템 초기화 중...
✅ 환경변수 로드 완료
📦 필수 모듈 로딩 중...
✅ 기본 모듈 로드
✅ 벡터 스토어 모듈 로드
✅ 후처리 모듈 로드
⚙️ 시스템 설정 중...
✅ 모델 설정 완료
📚 문서 데이터 준비 중...
✅ 3개 문서 생성 완료
🔍 벡터 인덱스 생성 중...
✅ 벡터 인덱스 생성 완료
🚀 고급 후처리 파이프라인 구성 중...
✅ 후처리 파이프라인 구성 완료
⚙️ 쿼리 엔진 생성 중...
✅ 쿼리 엔진 생성 완료

🎉 고급 RAG 시스템 준비 완료!

📋 기본 테스트:

🤔 질문: 정책 변화 핵심만 알려줘
🔍 검색 및 분석 중...

💡 답변:
----------------------------------------
Empty Response

📚 참조된 문서 (0개):
----------------------------------------

🔄 전체 테스트를 실행하시겠습니까? (y/n): 
🎯 종합 테스트 시작

[테스트 1/5]

🤔 질문: 정책 변화 핵심만 알려줘
🔍 검색 및 분석 중...

💡 답변:
----------------------------------------
Empty Response

📚 참조된 문서 (0개):
----------------------------------------


[테스트 2/5]

🤔 질문: 2025년에 새로 도입되는 AI 관련 정책은?
🔍 검색 및 분석 중...

💡 답변:
----------------------------------------
Empty Response

📚 참조된 문서 (0개):
----------------------------------------


[테스트 3/5]

🤔 질문: 디지털 전환과 관련된 주요 변화점은?
🔍 검색 및 분석 중...

💡 답변:
----------------------------------------
Empty Response

📚 참조된 문서 (0개):
----------------------------------------


[테스트

HyDE(Hypothetical Document Embeddings)란?
- HyDE는 쿼리 개선 기법 중 하나로, 가상의 답변 문서를 생성해서 검색 성능을 향상시키는 방법입니다.

- 작동 원리:
   - 사용자 질문 → LLM이 가상의 답변 문서 생성
   - 가상 문서를 벡터화해서 실제 문서와 유사도 검색
   - 원본 질문보다 더 구체적이고 상세한 검색이 가능

- 예시:

  - 원본 질문: "사내 AI 보안 준수 항목은?"
  - HyDE 생성 문서: "사내 AI 보안 준수 항목에는 데이터 암호화, 접근 권한 관리, 로그 모니터링, 개인정보 보호..."

In [13]:
# ==========================================
# HyDE RAG 시스템 (pandas 순환 임포트 오류 완전 해결)
# ==========================================

"""
pandas 오류 해결 방법:

1. 가상환경 완전 재설정:
   conda deactivate
   conda remove -n your_env --all
   conda create -n hyde_rag python=3.10
   conda activate hyde_rag

2. 패키지 순차 설치:
   pip install --no-cache-dir numpy==1.24.4
   pip install --no-cache-dir pandas==2.0.3
   pip install --no-cache-dir llama-index llama-index-llms-openai llama-index-embeddings-openai
   pip install --no-cache-dir python-dotenv
"""

import os
import sys
import warnings
import logging
import time
from pathlib import Path

# 환경 설정 (pandas 문제 우회)
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['CUDA_VISIBLE_DEVICES'] = ''  # GPU 사용 비활성화
warnings.filterwarnings('ignore')

print("🔧 시스템 초기화 중...")

# 환경변수 로드
try:
    from dotenv import load_dotenv
    load_dotenv()
    print("✅ 환경변수 로드 완료")
except ImportError:
    print("⚠️ python-dotenv 미설치")

# API 키 확인
if not os.getenv("OPENAI_API_KEY"):
    print("❌ OPENAI_API_KEY 필요")
    print("💡 .env 파일에 OPENAI_API_KEY=your_key 추가")
    sys.exit(1)

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# ==========================================
# 안전한 모듈 임포트
# ==========================================

print("📦 LlamaIndex 모듈 로딩 중...")

try:
    # 기본 모듈
    from llama_index.core import Settings, Document
    from llama_index.llms.openai import OpenAI
    from llama_index.embeddings.openai import OpenAIEmbedding
    print("✅ 기본 모듈 로드")
    
    # 벡터 스토어
    from llama_index.core import VectorStoreIndex
    print("✅ 벡터 스토어 모듈 로드")
    
    # HyDE 관련 모듈
    from llama_index.core.indices.query.query_transform import HyDEQueryTransform
    from llama_index.core.query_engine import TransformQueryEngine
    print("✅ HyDE 모듈 로드")
    
except ImportError as e:
    print(f"❌ 모듈 임포트 실패: {e}")
    print("💡 패키지 재설치 필요")
    sys.exit(1)

# ==========================================
# HyDE RAG 시스템 클래스
# ==========================================

class EnhancedHyDERAG:
    """pandas 문제 해결된 실무용 HyDE RAG 시스템"""
    
    def __init__(self, model_name='gpt-4o-mini'):
        """
        초기화
        Args:
            model_name: 사용할 LLM 모델
        """
        self.model_name = model_name
        self.index = None
        self.hyde_query_engine = None
        self.normal_query_engine = None
        
        # 샘플 문서 데이터 (파일 시스템 우회)
        self.sample_documents = self._create_sample_documents()
        
        # 모델 설정
        self._setup_models()
        
    def _create_sample_documents(self):
        """샘플 문서 데이터 생성 (파일 I/O 없이)"""
        return [
            {
                "title": "사내 AI 보안 정책",
                "content": """
                AI 보안 준수 항목 및 가이드라인:

                1. 핵심 보안 준수 항목
                - AI 모델 접근 권한 관리: 역할 기반 접근 제어 적용
                - 데이터 암호화: 학습 데이터 및 모델 가중치 암호화 저장
                - 모델 버전 관리: 승인된 모델만 운영 환경 배포
                - 입출력 데이터 검증: 악성 입력 탐지 및 차단

                2. AI 모델 보안 요구사항
                - 모델 추론 시 로그 기록 의무화
                - 민감 정보 포함 여부 사전 스캔
                - 적대적 공격(Adversarial Attack) 방어 메커니즘
                - 모델 편향성 정기 검사 및 보고

                3. 데이터 보안 정책
                - 개인정보 비식별화 처리 필수
                - 학습 데이터 보관 기간 제한 (최대 3년)
                - 데이터 접근 이력 추적 및 감사
                - 클라우드 저장 시 국내 데이터센터 사용

                4. 직원 교육 및 인증
                - 분기별 AI 보안 교육 이수 의무
                - AI 윤리 가이드라인 준수 서약
                - 보안 인시던트 발생 시 즉시 보고 체계
                """
            },
            {
                "title": "개인정보보호 정책",
                "content": """
                데이터 개인정보 보호 정책 세부 사항:

                1. 개인정보 수집 및 이용 원칙
                - 목적 외 사용 금지: 명시된 목적으로만 활용
                - 최소 수집 원칙: 필요한 최소한의 정보만 수집
                - 동의 기반 수집: 명확한 동의 절차 필수
                - 수집 정보 고지: 수집 항목 및 목적 사전 공지

                2. 개인정보 처리 보안 조치
                - 개인정보 암호화 저장 (AES-256 이상)
                - 접근 권한 최소화 및 정기 검토
                - 개인정보 처리 시스템 분리 운영
                - 개인정보 이용 내역 로그 보관 (3년)

                3. 개인정보 보관 및 폐기
                - 보유 기간 경과 시 즉시 파기
                - 복구 불가능한 방법으로 완전 삭제
                - 파기 과정 문서화 및 증빙 보관
                - 개인정보 처리 현황 정기 점검

                4. 정보 주체의 권리 보장
                - 개인정보 열람권 보장
                - 정정 및 삭제 요구권 즉시 처리
                - 처리 정지 요구 시 14일 내 조치
                - 손해 발생 시 배상 책임 이행

                5. 개인정보 유출 대응 체계
                - 유출 발견 시 즉시 신고 (72시간 내)
                - 정보주체 개별 통지 및 조치 안내
                - 유출 원인 분석 및 재발 방지 대책
                - 관련 기관 협조 및 후속 조치
                """
            },
            {
                "title": "클라우드 서비스 보안 가이드",
                "content": """
                클라우드 서비스 보안 가이드라인:

                1. 클라우드 서비스 선정 기준
                - 국제 보안 인증 보유 (ISO 27001, SOC 2)
                - 국내 법규 준수 (개인정보보호법, 정보통신망법)
                - 데이터 주권 보장 (국내 데이터센터 운영)
                - SLA 99.9% 이상 가용성 보장

                2. 클라우드 보안 아키텍처
                - 네트워크 분리: VPC 및 서브넷 분리 구성
                - 접근 제어: IAM 기반 세밀한 권한 관리
                - 데이터 암호화: 전송 구간 및 저장 시 암호화
                - 모니터링: 실시간 보안 이벤트 감시

                3. 클라우드 데이터 보안
                - 중요 데이터 분류 및 라벨링
                - 데이터 백업 및 복구 계획 수립
                - 데이터 이동 시 보안 프로토콜 적용
                - 임시 데이터 자동 삭제 정책

                4. 클라우드 서비스 운영 보안
                - 정기 보안 취약점 점검
                - 패치 관리 및 업데이트 정책
                - 인시던트 대응 절차 수립
                - 벤더 보안 수준 정기 평가

                5. 클라우드 컴플라이언스
                - 규제 요구사항 준수 점검
                - 감사 로그 수집 및 보관
                - 제3자 보안 평가 정기 실시
                - 클라우드 계약 보안 조항 포함
                """
            },
            {
                "title": "직원 보안 교육 프로그램",
                "content": """
                직원 교육 프로그램 내용 및 운영 방안:

                1. 정보보안 기초 교육
                - 정보보안 개념 및 중요성 인식
                - 사내 보안 정책 및 절차 숙지
                - 개인정보보호 원칙 및 의무사항
                - 보안 인시던트 신고 체계

                2. 사이버 위협 대응 교육
                - 피싱 메일 및 악성코드 식별법
                - 사회공학기법 인지 및 대응 방법
                - 안전한 인터넷 사용 가이드라인
                - 모바일 보안 및 BYOD 정책

                3. AI 및 데이터 보안 교육
                - AI 시스템 보안 위험 요소
                - 데이터 분류 및 취급 방법
                - 머신러닝 모델 보안 고려사항
                - AI 윤리 및 편향성 방지

                4. 교육 운영 체계
                - 신입사원 필수 교육 (입사 1개월 내)
                - 전체 직원 정기 교육 (분기별)
                - 부서별 맞춤형 교육 프로그램
                - 온라인 교육 플랫폼 운영

                5. 교육 효과 측정 및 개선
                - 교육 이수 현황 추적 관리
                - 보안 인식 수준 정기 평가
                - 모의 침투 테스트 실시
                - 교육 내용 지속적 업데이트

                6. 보안 문화 조성 활동
                - 보안 우수사례 공유 및 포상
                - 보안 캠페인 및 인식 제고 활동
                - 보안 관련 세미나 및 컨퍼런스 참여
                - 보안 담당자와의 정기 소통 채널
                """
            }
        ]
    
    def _setup_models(self):
        """모델 및 설정 초기화"""
        try:
            # LLM 설정 (안정적인 모델 사용)
            llm = OpenAI(
                model=self.model_name,
                temperature=0.1,
                max_tokens=1500,
                timeout=60
            )
            
            # 임베딩 모델 설정 (가벼운 모델)
            embed_model = OpenAIEmbedding(
                model="text-embedding-3-small",
                dimensions=512,
                timeout=60
            )
            
            # 전역 설정
            Settings.llm = llm
            Settings.embed_model = embed_model
            
            logger.info(f"✅ 모델 설정 완료: {self.model_name}")
            
        except Exception as e:
            logger.error(f"❌ 모델 설정 실패: {e}")
            raise
    
    def load_documents(self):
        """문서 로드 및 인덱스 생성 (파일 시스템 우회)"""
        try:
            logger.info("📚 문서 생성 중...")
            
            # Document 객체 생성
            documents = []
            for doc_data in self.sample_documents:
                doc = Document(
                    text=doc_data["content"],
                    metadata={"title": doc_data["title"]}
                )
                documents.append(doc)
            
            logger.info(f"✅ {len(documents)}개 문서 생성 완료")
            
            # 인덱스 생성
            logger.info("🔍 벡터 인덱스 생성 중...")
            self.index = VectorStoreIndex.from_documents(
                documents,
                show_progress=False  # 안정성을 위해 비활성화
            )
            logger.info("✅ 인덱스 생성 완료")
            
        except Exception as e:
            logger.error(f"❌ 문서 로딩 실패: {e}")
            raise
    
    def setup_query_engines(self, similarity_top_k=5, hyde_include_original=True):
        """쿼리 엔진 설정"""
        if not self.index:
            raise ValueError("먼저 load_documents()를 실행해주세요")
        
        try:
            logger.info("⚙️ 쿼리 엔진 설정 중...")
            
            # 일반 쿼리 엔진
            self.normal_query_engine = self.index.as_query_engine(
                similarity_top_k=similarity_top_k,
                response_mode="tree_summarize"
            )
            
            # HyDE 변환기 설정
            hyde_transform = HyDEQueryTransform(
                include_original=hyde_include_original,
                llm=Settings.llm  # 명시적 LLM 지정
            )
            
            # HyDE 쿼리 엔진
            self.hyde_query_engine = TransformQueryEngine(
                query_engine=self.normal_query_engine,
                query_transform=hyde_transform
            )
            
            logger.info("✅ 쿼리 엔진 설정 완료")
            
        except Exception as e:
            logger.error(f"❌ 쿼리 엔진 설정 실패: {e}")
            raise
    
    def get_hyde_transformed_query(self, question: str):
        """HyDE 변환된 쿼리 확인"""
        try:
            print(f"\n🤔 원본 질문: {question}")
            print("=" * 50)
            
            # HyDE 변환기 생성
            hyde_transform = HyDEQueryTransform(
                include_original=False,
                llm=Settings.llm
            )
            
            # 변환 실행
            print("🔄 HyDE 변환 중...")
            transformed = hyde_transform.run(question)
            
            print("\n📄 HyDE 생성 가상 문서:")
            print("-" * 40)
            print(transformed.query_str)
            
            return transformed.query_str
            
        except Exception as e:
            logger.error(f"❌ HyDE 변환 실패: {e}")
            print(f"❌ HyDE 변환 오류: {e}")
            return None
    
    def query_with_comparison(self, question: str, verbose=True):
        """HyDE와 일반 검색 비교"""
        if not self.hyde_query_engine or not self.normal_query_engine:
            raise ValueError("먼저 setup_query_engines()를 실행해주세요")
        
        results = {}
        
        try:
            if verbose:
                print(f"\n{'='*70}")
                print(f"🤔 질문: {question}")
                print(f"{'='*70}")
            
            # 일반 검색
            print("🔍 일반 검색 실행 중...")
            start_time = time.time()
            normal_response = self.normal_query_engine.query(question)
            normal_time = time.time() - start_time
            
            # HyDE 검색
            print("🎯 HyDE 검색 실행 중...")
            start_time = time.time()
            hyde_response = self.hyde_query_engine.query(question)
            hyde_time = time.time() - start_time
            
            results = {
                'question': question,
                'normal_response': str(normal_response),
                'normal_time': normal_time,
                'hyde_response': str(hyde_response),
                'hyde_time': hyde_time
            }
            
            if verbose:
                print(f"\n📋 일반 검색 결과 (소요시간: {normal_time:.2f}초):")
                print("-" * 50)
                print(str(normal_response))
                
                print(f"\n🎯 HyDE 검색 결과 (소요시간: {hyde_time:.2f}초):")
                print("-" * 50)
                print(str(hyde_response))
                
                print(f"\n⚡ 성능 비교:")
                print("-" * 30)
                print(f"   일반 검색: {normal_time:.2f}초")
                print(f"   HyDE 검색: {hyde_time:.2f}초")
                print(f"   시간 차이: {hyde_time - normal_time:+.2f}초")
                
                # 응답 품질 비교
                print(f"\n📊 응답 분석:")
                print("-" * 30)
                print(f"   일반 응답 길이: {len(str(normal_response))}자")
                print(f"   HyDE 응답 길이: {len(str(hyde_response))}자")
            
            return results
            
        except Exception as e:
            logger.error(f"❌ 쿼리 실행 실패: {e}")
            print(f"❌ 쿼리 실행 오류: {e}")
            raise

# ==========================================
# 사용 예시 및 테스트 함수
# ==========================================

def comprehensive_test():
    """종합 테스트 실행"""
    try:
        print("🚀 HyDE RAG 시스템 초기화")
        print("=" * 60)
        
        # 시스템 초기화
        rag = EnhancedHyDERAG(model_name='gpt-4o-mini')
        
        # 문서 로드
        rag.load_documents()
        
        # 쿼리 엔진 설정
        rag.setup_query_engines(similarity_top_k=5)
        
        # 테스트 질문들
        test_questions = [
            "사내 AI 보안 준수 항목은?",
            "데이터 개인정보 보호 정책은 무엇인가요?",
            "클라우드 서비스 보안 가이드라인은?",
            "직원 교육 프로그램 내용은?"
        ]
        
        print("\n🔍 HyDE 변환 테스트")
        print("=" * 60)
        
        # HyDE 변환 확인 (처음 2개 질문)
        for i, question in enumerate(test_questions[:2], 1):
            print(f"\n[HyDE 변환 테스트 {i}]")
            rag.get_hyde_transformed_query(question)
            print("\n" + "="*60)
        
        print("\n📊 검색 성능 비교 테스트")
        print("=" * 60)
        
        # 비교 검색 실행
        for i, question in enumerate(test_questions, 1):
            print(f"\n[비교 테스트 {i}/{len(test_questions)}]")
            results = rag.query_with_comparison(question, verbose=True)
            print("\n" + "="*80)
        
        print("\n🎉 모든 테스트 완료!")
        return rag
        
    except Exception as e:
        logger.error(f"❌ 테스트 실행 실패: {e}")
        print(f"❌ 테스트 오류: {e}")
        return None

def interactive_mode(rag_system):
    """대화형 모드"""
    if not rag_system:
        print("❌ RAG 시스템이 초기화되지 않았습니다.")
        return
    
    print("\n💬 대화형 HyDE RAG 모드")
    print("-" * 40)
    print("명령어:")
    print("  'hyde [질문]' - HyDE 변환 확인")
    print("  'compare [질문]' - 일반 vs HyDE 비교")
    print("  'quit' - 종료")
    print("-" * 40)
    
    while True:
        try:
            user_input = input("\n입력: ").strip()
            
            if user_input.lower() in ['quit', 'exit', '종료']:
                print("👋 대화를 종료합니다.")
                break
            
            if user_input.startswith('hyde '):
                question = user_input[5:].strip()
                if question:
                    rag_system.get_hyde_transformed_query(question)
                
            elif user_input.startswith('compare '):
                question = user_input[8:].strip()
                if question:
                    rag_system.query_with_comparison(question, verbose=True)
                    
            elif user_input:
                # 기본적으로 비교 검색 실행
                rag_system.query_with_comparison(user_input, verbose=True)
                
        except KeyboardInterrupt:
            print("\n\n👋 대화를 종료합니다.")
            break
        except Exception as e:
            print(f"❌ 오류: {e}")

# ==========================================
# 메인 실행부
# ==========================================

if __name__ == "__main__":
    print("🎯 HyDE RAG 시스템 시작")
    print("=" * 50)
    
    # 전체 테스트 실행
    rag_system = comprehensive_test()
    
    # 대화형 모드 옵션
    if rag_system:
        print("\n💬 대화형 모드를 시작하시겠습니까? (y/n): ", end="")
        try:
            choice = input().strip().lower()
            if choice in ['y', 'yes', '예']:
                interactive_mode(rag_system)
        except:
            pass
    
    print("\n🎊 프로그램 종료")

# ==========================================
# 개별 실행을 위한 간단한 래퍼
# ==========================================

def quick_test():
    """빠른 테스트용 함수"""
    try:
        # 초기화
        rag = EnhancedHyDERAG()
        rag.load_documents()
        rag.setup_query_engines()
        
        # HyDE 변환 확인
        print("🔄 HyDE 변환 테스트:")
        rag.get_hyde_transformed_query("사내 AI 보안 준수 항목은?")
        
        # 비교 검색
        print("\n📊 검색 비교 테스트:")
        results = rag.query_with_comparison("사내 AI 보안 준수 항목은?")
        
        return rag
        
    except Exception as e:
        print(f"❌ 빠른 테스트 실패: {e}")
        return None

# 사용법:
# 1. 전체 테스트: 코드 실행
# 2. 빠른 테스트: quick_test() 함수 호출

2025-09-13 01:35:05,592 - INFO - ✅ 모델 설정 완료: gpt-4o-mini
2025-09-13 01:35:05,592 - INFO - 📚 문서 생성 중...
2025-09-13 01:35:05,594 - INFO - ✅ 4개 문서 생성 완료
2025-09-13 01:35:05,596 - INFO - 🔍 벡터 인덱스 생성 중...


🔧 시스템 초기화 중...
✅ 환경변수 로드 완료
📦 LlamaIndex 모듈 로딩 중...
✅ 기본 모듈 로드
✅ 벡터 스토어 모듈 로드
✅ HyDE 모듈 로드
🎯 HyDE RAG 시스템 시작
🚀 HyDE RAG 시스템 초기화


2025-09-13 01:35:06,183 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-09-13 01:35:06,192 - INFO - ✅ 인덱스 생성 완료
2025-09-13 01:35:06,194 - INFO - ⚙️ 쿼리 엔진 설정 중...
2025-09-13 01:35:06,196 - INFO - ✅ 쿼리 엔진 설정 완료



🔍 HyDE 변환 테스트

[HyDE 변환 테스트 1]

🤔 원본 질문: 사내 AI 보안 준수 항목은?
🔄 HyDE 변환 중...


2025-09-13 01:35:14,937 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...


📋 일반 검색 결과 (소요시간: 8.09초):
--------------------------------------------------
클라우드 서비스 보안 가이드라인은 다음과 같습니다:

1. **클라우드 서비스 선정 기준**
   - 국제 보안 인증 보유 (ISO 27001, SOC 2)
   - 국내 법규 준수 (개인정보보호법, 정보통신망법)
   - 데이터 주권 보장 (국내 데이터센터 운영)
   - SLA 99.9% 이상 가용성 보장

2. **클라우드 보안 아키텍처**
   - 네트워크 분리: VPC 및 서브넷 분리 구성
   - 접근 제어: IAM 기반 세밀한 권한 관리
   - 데이터 암호화: 전송 구간 및 저장 시 암호화
   - 모니터링: 실시간 보안 이벤트 감시

3. **클라우드 데이터 보안**
   - 중요 데이터 분류 및 라벨링
   - 데이터 백업 및 복구 계획 수립
   - 데이터 이동 시 보안 프로토콜 적용
   - 임시 데이터 자동 삭제 정책

4. **클라우드 서비스 운영 보안**
   - 정기 보안 취약점 점검
   - 패치 관리 및 업데이트 정책
   - 인시던트 대응 절차 수립
   - 벤더 보안 수준 정기 평가

5. **클라우드 컴플라이언스**
   - 규제 요구사항 준수 점검
   - 감사 로그 수집 및 보관
   - 제3자 보안 평가 정기 실시
   - 클라우드 계약 보안 조항 포함

🎯 HyDE 검색 결과 (소요시간: 18.16초):
--------------------------------------------------
클라우드 서비스 보안 가이드라인은 다음과 같습니다:

1. 클라우드 서비스 선정 기준
   - 국제 보안 인증 보유 (ISO 27001, SOC 2)
   - 국내 법규 준수 (개인정보보호법, 정보통신망법)
   - 데이터 주권 보장 (국내 데이터센터 운영)
   - SLA 99.9% 이상 가용성 보장

2. 클라우드 보안 아키텍처
   - 네트워크 분리: VPC 및 서브넷 분리 구성
  

2025-09-13 01:36:16,758 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-09-13 01:36:24,543 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


🎯 HyDE 검색 실행 중...


2025-09-13 01:36:33,030 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-09-13 01:36:33,357 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-09-13 01:36:36,062 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-09-13 01:36:44,133 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



📋 일반 검색 결과 (소요시간: 8.42초):
--------------------------------------------------
직원 교육 프로그램은 다음과 같은 내용으로 구성됩니다:

1. 정보보안 기초 교육
   - 정보보안 개념 및 중요성 인식
   - 사내 보안 정책 및 절차 숙지
   - 개인정보보호 원칙 및 의무사항
   - 보안 인시던트 신고 체계

2. 사이버 위협 대응 교육
   - 피싱 메일 및 악성코드 식별법
   - 사회공학기법 인지 및 대응 방법
   - 안전한 인터넷 사용 가이드라인
   - 모바일 보안 및 BYOD 정책

3. AI 및 데이터 보안 교육
   - AI 시스템 보안 위험 요소
   - 데이터 분류 및 취급 방법
   - 머신러닝 모델 보안 고려사항
   - AI 윤리 및 편향성 방지

4. 교육 운영 체계
   - 신입사원 필수 교육 (입사 1개월 내)
   - 전체 직원 정기 교육 (분기별)
   - 부서별 맞춤형 교육 프로그램
   - 온라인 교육 플랫폼 운영

5. 교육 효과 측정 및 개선
   - 교육 이수 현황 추적 관리
   - 보안 인식 수준 정기 평가
   - 모의 침투 테스트 실시
   - 교육 내용 지속적 업데이트

6. 보안 문화 조성 활동
   - 보안 우수사례 공유 및 포상
   - 보안 캠페인 및 인식 제고 활동
   - 보안 관련 세미나 및 컨퍼런스 참여
   - 보안 담당자와의 정기 소통 채널

🎯 HyDE 검색 결과 (소요시간: 19.59초):
--------------------------------------------------
직원 교육 프로그램은 다음과 같은 내용으로 구성됩니다:

1. 정보보안 기초 교육
   - 정보보안 개념 및 중요성 인식
   - 사내 보안 정책 및 절차 숙지
   - 개인정보보호 원칙 및 의무사항
   - 보안 인시던트 신고 체계

2. 사이버 위협 대응 교육
   - 피싱 메일 및 악성코드 식별법
   - 사회공학기법 인지 및 대응 방법
   

In [14]:
# eos